In [1]:
import numpy as np
import time
from tensorflow.examples.tutorials.mnist import input_data

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Reshape
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D, LSTM, TimeDistributed
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
class ElapsedTimer(object):
    def __init__(self):
        self.start_time = time.time()
    def elapsed(self,sec):
        if sec < 60:
            return str(sec) + " sec"
        elif sec < (60 * 60):
            return str(sec / 60) + " min"
        else:
            return str(sec / (60 * 60)) + " hr"
    def elapsed_time(self):
        print("Elapsed: %s " % self.elapsed(time.time() - self.start_time) )

In [3]:
class DCGAN(object):
    def __init__(self, img_rows=28, img_cols=28, channel=1):
        self.img_rows = img_rows
        self.img_cols = img_cols
        self.channel = channel
        self.D = None   # discriminator
        self.G = None   # generator
        self.RG = None  # recurrent generator
        self.DM = None  # discriminator model
        self.AM = None  # adversarial model
        self.RAM = None  # recurrent adversarial model

    # (W−F+2P)/S+1
    def discriminator(self):
        if self.D:
            return self.D
        self.D = Sequential()
        depth = 64
        dropout = 0.4
        # In: 28 x 28 x 1, depth = 1
        # Out: 14 x 14 x 1, depth=64
        input_shape = (self.img_rows, self.img_cols, self.channel)
        self.D.add(Conv2D(depth*1, 5, strides=2, input_shape=input_shape,\
            padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(depth*2, 5, strides=2, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(depth*4, 5, strides=2, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(depth*8, 5, strides=1, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        # Out: 1-dim probability
        self.D.add(Flatten())
        self.D.add(Dense(1))
        self.D.add(Activation('sigmoid'))
        self.D.summary()
        return self.D

    def generator(self):
        if self.G:
            return self.G
        self.G = Sequential()
        dropout = 0.4
        depth = 64+64+64+64
        dim = 7
        # In: 100
        # Out: dim x dim x depth
        self.G.add(Dense(dim*dim*depth, input_dim=100))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))
        self.G.add(Reshape((dim, dim, depth)))
        self.G.add(Dropout(dropout))

        # In: dim x dim x depth
        # Out: 2*dim x 2*dim x depth/2
        self.G.add(UpSampling2D())
        self.G.add(Conv2DTranspose(int(depth/2), 5, padding='same'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))

        self.G.add(UpSampling2D())
        self.G.add(Conv2DTranspose(int(depth/4), 5, padding='same'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))

        self.G.add(Conv2DTranspose(int(depth/8), 5, padding='same'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))

        # Out: 28 x 28 x 1 grayscale image [0.0,1.0] per pix
        self.G.add(Conv2DTranspose(1, 5, padding='same'))
        self.G.add(Activation('sigmoid'))
        self.G.summary()
        return self.G
    
    def recurrent_generator(self):
        if self.RG:
            return self.RG
        dropout = 0.4
        depth = 64+64+64+64
        dim = 14
        
        self.RG = Sequential()
        self.RG.add(LSTM(100, input_shape=(None, 100),
                         return_sequences=True, activation='tanh', recurrent_activation='hard_sigmoid'))
        self.RG.add(LSTM(100, return_sequences=False, activation='tanh', recurrent_activation='hard_sigmoid'))
        #self.RG.add(LSTM(10, return_sequences=False, activation='tanh', recurrent_activation='hard_sigmoid'))

        self.RG.add(Dense(dim*dim*depth))
        self.RG.add(BatchNormalization(momentum=0.9))
        self.RG.add(Activation('relu'))

        self.RG.add(Reshape((dim, dim, depth)))
        self.RG.add(Dropout(dropout))
        
        self.RG.add(UpSampling2D())
        self.RG.add(Conv2DTranspose(1, 5, padding='same'))
        self.RG.add(Activation('sigmoid'))
        
        self.RG.summary()
        return self.RG

    def discriminator_model(self):
        if self.DM:
            return self.DM
        optimizer = RMSprop(lr=0.0002, decay=6e-8)
        self.DM = Sequential()
        self.DM.add(self.discriminator())
        self.DM.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        return self.DM

    def adversarial_model(self):
        if self.AM:
            return self.AM
        optimizer = RMSprop(lr=0.0001, decay=3e-8)
        self.AM = Sequential()
        self.AM.add(self.generator())
        self.AM.add(self.discriminator())
        self.AM.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        return self.AM
    
    def recurrent_adversarial_model(self):
        if self.RAM:
            return self.RAM
        optimizer = RMSprop(lr=0.0001, decay=3e-8)
        self.RAM = Sequential()
        self.RAM.add(self.recurrent_generator())
        self.RAM.add(self.discriminator())
        self.RAM.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        return self.RAM

In [4]:
class MNIST_DCGAN(object):
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channel = 1

        self.x_train = input_data.read_data_sets("mnist", one_hot=True).train.images
        self.x_train = self.x_train.reshape(-1, self.img_rows, self.img_cols, 1).astype(np.float32)

        self.DCGAN = DCGAN()
        self.discriminator =  self.DCGAN.discriminator_model()
        
        recurrent = False
        if recurrent:
            self.adversarial = self.DCGAN.recurrent_adversarial_model()
            self.generator = self.DCGAN.recurrent_generator()
        else:
            self.adversarial = self.DCGAN.adversarial_model()
            self.generator = self.DCGAN.generator()

    def train(self, train_steps=2000, batch_size=256, save_interval=0):
        noise_input = None
        if save_interval > 0:
            noise_input = np.random.uniform(-1.0, 1.0, size=[16, 100])
        for i in range(train_steps):
            images_train = self.x_train[np.random.randint(0,
                self.x_train.shape[0], size=batch_size), :, :, :]
            noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
            images_fake = self.generator.predict(noise)
            x = np.concatenate((images_train, images_fake))
            y = np.ones([2*batch_size, 1])
            y[batch_size:, :] = 0
            d_loss = self.discriminator.train_on_batch(x, y)

            y = np.ones([batch_size, 1])
            noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
            a_loss = self.adversarial.train_on_batch(noise, y)
            log_mesg = "%d: [D loss: %f, acc: %f]" % (i, d_loss[0], d_loss[1])
            log_mesg = "%s  [A loss: %f, acc: %f]" % (log_mesg, a_loss[0], a_loss[1])
            print(log_mesg)
            if save_interval > 0:
                if (i+1)%save_interval==0:
                    self.plot_images(save2file=True, samples=noise_input.shape[0], noise=noise_input, step=(i+1))

    def plot_images(self, save2file=False, fake=True, samples=16, noise=None, step=0):
        filename = 'mnist.png'
        if fake:
            if noise is None:
                noise = np.random.uniform(-1.0, 1.0, size=[samples, 100])
            else:
                filename = "mnist_%d.png" % step
            images = self.generator.predict(noise)
        else:
            i = np.random.randint(0, self.x_train.shape[0], samples)
            images = self.x_train[i, :, :, :]

        plt.figure(figsize=(10,10))
        for i in range(images.shape[0]):
            plt.subplot(4, 4, i+1)
            image = images[i, :, :, :]
            image = np.reshape(image, [self.img_rows, self.img_cols])
            plt.imshow(image, cmap='gray')
            plt.axis('off')
        plt.tight_layout()
        if save2file:
            plt.savefig(filename)
            plt.close('all')
        else:
            plt.show()

In [5]:
if __name__ == '__main__':
    mnist_dcgan = MNIST_DCGAN()
    timer = ElapsedTimer()
    mnist_dcgan.train(train_steps=10000, batch_size=256, save_interval=100)
    timer.elapsed_time()
    mnist_dcgan.plot_images(fake=True)
    mnist_dcgan.plot_images(fake=False, save2file=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 64)        1664      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 64)        0 

14: [D loss: 0.453520, acc: 0.800781]  [A loss: 6.864108, acc: 0.000000]
15: [D loss: 0.425948, acc: 0.812500]  [A loss: 5.130146, acc: 0.000000]
16: [D loss: 0.640605, acc: 0.707031]  [A loss: 10.101437, acc: 0.000000]
17: [D loss: 0.575113, acc: 0.787109]  [A loss: 1.449203, acc: 0.160156]
18: [D loss: 1.416567, acc: 0.511719]  [A loss: 15.433580, acc: 0.000000]
19: [D loss: 2.618127, acc: 0.500000]  [A loss: 2.011896, acc: 0.074219]
20: [D loss: 0.800570, acc: 0.654297]  [A loss: 9.290813, acc: 0.000000]
21: [D loss: 0.519102, acc: 0.802734]  [A loss: 0.964047, acc: 0.421875]
22: [D loss: 1.458806, acc: 0.498047]  [A loss: 11.916225, acc: 0.000000]
23: [D loss: 0.749651, acc: 0.677734]  [A loss: 1.073137, acc: 0.273438]
24: [D loss: 1.657751, acc: 0.488281]  [A loss: 10.938141, acc: 0.000000]
25: [D loss: 0.815179, acc: 0.652344]  [A loss: 1.187152, acc: 0.175781]
26: [D loss: 1.621934, acc: 0.484375]  [A loss: 8.687269, acc: 0.000000]
27: [D loss: 0.681097, acc: 0.720703]  [A loss:

126: [D loss: 0.801225, acc: 0.642578]  [A loss: 0.027930, acc: 1.000000]
127: [D loss: 1.991572, acc: 0.500000]  [A loss: 1.372067, acc: 0.054688]
128: [D loss: 0.814242, acc: 0.550781]  [A loss: 2.893261, acc: 0.000000]
129: [D loss: 0.679052, acc: 0.587891]  [A loss: 1.355523, acc: 0.097656]
130: [D loss: 1.005040, acc: 0.509766]  [A loss: 5.451816, acc: 0.000000]
131: [D loss: 0.766486, acc: 0.617188]  [A loss: 0.049094, acc: 1.000000]
132: [D loss: 1.689599, acc: 0.500000]  [A loss: 2.490917, acc: 0.000000]
133: [D loss: 0.700623, acc: 0.546875]  [A loss: 1.538088, acc: 0.054688]
134: [D loss: 0.988866, acc: 0.492188]  [A loss: 5.317080, acc: 0.000000]
135: [D loss: 0.770678, acc: 0.621094]  [A loss: 0.041748, acc: 1.000000]
136: [D loss: 1.883667, acc: 0.500000]  [A loss: 2.514209, acc: 0.000000]
137: [D loss: 0.696938, acc: 0.568359]  [A loss: 1.275826, acc: 0.152344]
138: [D loss: 0.982794, acc: 0.511719]  [A loss: 5.275397, acc: 0.000000]
139: [D loss: 0.799707, acc: 0.626953]

237: [D loss: 1.838306, acc: 0.509766]  [A loss: 8.460901, acc: 0.000000]
238: [D loss: 0.822926, acc: 0.654297]  [A loss: 0.084982, acc: 0.984375]
239: [D loss: 2.257688, acc: 0.505859]  [A loss: 7.490257, acc: 0.000000]
240: [D loss: 0.832847, acc: 0.630859]  [A loss: 0.115731, acc: 0.968750]
241: [D loss: 2.247760, acc: 0.500000]  [A loss: 7.293711, acc: 0.000000]
242: [D loss: 0.752559, acc: 0.671875]  [A loss: 0.095449, acc: 0.980469]
243: [D loss: 2.221474, acc: 0.501953]  [A loss: 6.873390, acc: 0.000000]
244: [D loss: 0.722663, acc: 0.675781]  [A loss: 0.290198, acc: 0.871094]
245: [D loss: 1.817181, acc: 0.509766]  [A loss: 8.629904, acc: 0.000000]
246: [D loss: 1.010552, acc: 0.638672]  [A loss: 0.005601, acc: 1.000000]
247: [D loss: 3.421878, acc: 0.500000]  [A loss: 3.038924, acc: 0.031250]
248: [D loss: 1.615841, acc: 0.542969]  [A loss: 9.523657, acc: 0.000000]
249: [D loss: 0.990389, acc: 0.654297]  [A loss: 0.006151, acc: 1.000000]
250: [D loss: 3.780322, acc: 0.500000]

348: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
349: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
350: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
351: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
352: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
353: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
354: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
355: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
356: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
357: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
358: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
359: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
360: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
361: [D loss: 7.971192, acc: 0.500000]

...

3544: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3545: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3546: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3547: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3548: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3549: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3550: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3551: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3552: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3553: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3554: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3555: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3556: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3557: [D loss: 7.971192, 

3654: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3655: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3656: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3657: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3658: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3659: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3660: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3661: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3662: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3663: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3664: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3665: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3666: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3667: [D loss: 7.971192, 

3764: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3765: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3766: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3767: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3768: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3769: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3770: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3771: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3772: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3773: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3774: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3775: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3776: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3777: [D loss: 7.971192, 

3874: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3875: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3876: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3877: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3878: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3879: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3880: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3881: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3882: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3883: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3884: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3885: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]
3886: [D loss: 7.971192, acc: 0.500000]  [A loss: 0.000000, acc: 1.000000]


KeyboardInterrupt: 